In [1]:
import sympy as sp
import scipy as sc
import numpy as np
from plotly import graph_objects as go

from typing import List, Tuple, Dict, Callable

#### Mausqui herramientas misteriosas

In [2]:
def numeric_integration(f:callable, integration_limits: List[Tuple]):
    return sc.integrate.nquad(f, integration_limits)

# 1.

# 2. Consider the parametrization for the Torus $T$ given by:

$$\vec{X} = \left( (a + rcos(u))cos(v),\;\; (a + rcos(u))sin(v),\;\; rsin(u)  \right)$$

$$U = \left\{ (\theta, \phi) \in R^{2} | 0 < u < 2\pi,\;\; 0 < v < 2\pi  \right\}$$

with $0 < r < a$

- Is $T$ a regular surface? Justify your answer completely

In [3]:
u, v, a, r = sp.symbols('u v a r', real = True, positive = True)
ulimits = (0, 2*sp.pi)
vlimits = (0, 2*sp.pi)

x = sp.Matrix([(a + r*sp.cos(u))*sp.cos(v), (a + r*sp.cos(u))*sp.sin(v), r*sp.sin(u)])
x

Matrix([
[(a + r*cos(u))*cos(v)],
[(a + r*cos(u))*sin(v)],
[             r*sin(u)]])

- Evaluate the tangent vectors on T

In [4]:
# Calculate the tangent vectors to the surface
xu = x.diff(u)
xu

Matrix([
[-r*sin(u)*cos(v)],
[-r*sin(u)*sin(v)],
[        r*cos(u)]])

In [5]:
xv = x.diff(v)
xv

Matrix([
[-(a + r*cos(u))*sin(v)],
[ (a + r*cos(u))*cos(v)],
[                     0]])

- Evaluate the coeficients of the first fundamental form

In [6]:
# Evaluate the coefficients of the first fundamental form
E_1 = xu.dot(xu).simplify()
E_1

r**2

In [7]:
F_1 = xu.dot(xv)
F_1

0

In [8]:
G_1 = xv.dot(xv).simplify()
G_1

(a + r*cos(u))**2

- Which is the metric tensor for the Torus?

In [9]:
# Calculate the metric tensors for the surface
g = sp.Matrix([[E_1, F_1], [F_1, G_1]])
g

Matrix([
[r**2,                 0],
[   0, (a + r*cos(u))**2]])

- Which is the area element for the surface?

In [10]:
# Area element for the surface
ds = sp.sqrt(E_1*G_1 - F_1**2)
ds

r*Abs(a + r*cos(u))

- Evaluate the area of the surface.

In [11]:
# calculate the integral
I_1 = sp.integrate(r*(a + r*sp.cos(u)), (u, *ulimits), (v, *vlimits))
I_1

4*pi**2*a*r

- Evaluate the second fundamental form of the surface

In [12]:
# Second fundamental form of the surface
xuu = xu.diff(u)
xuv = xu.diff(v)
xvv = xv.diff(v)

In [13]:
N = (xu.cross(xv)/xu.cross(xv).norm())

E_12 = N.dot(xuu).simplify()
F_12 = N.dot(xuv).simplify()
G_12 = N.dot(xvv).simplify()

In [14]:
E_12

r*(a + r*cos(u))/Abs(a + r*cos(u))

In [15]:
F_12

0

In [16]:
G_12

(a + r*cos(u))**2*cos(u)/Abs(a + r*cos(u))

- Evaluate and plot the principal curvatures of the surface

In [17]:
dNP = -sp.Matrix([[E_12, F_12], [F_12, G_12]]) * sp.Matrix([[E_1, F_1], [F_1, G_1]]).inv()
K = sp.det(dNP)
K

cos(u)/(a*r + r**2*cos(u))

In [18]:
H = dNP.trace()/2
H

-(a + r*cos(u))**2*cos(u)/(2*(a**2 + 2*a*r*cos(u) + r**2*cos(u)**2)*Abs(a + r*cos(u))) - (a + r*cos(u))/(2*r*Abs(a + r*cos(u)))

In [19]:
k1 = (H + sp.sqrt(H**2 - K)).simplify()
k1

-cos(u)/Abs(a + r*cos(u))

In [20]:
k2 = (H - sp.sqrt(H**2 - K)).simplify()
k2

-(a + r*cos(u))/(r*Abs(a + r*cos(u)))

In order to be able to plot this, we gotta replace the $a, r$ constants with some values. Let's use $a = 2$ and $r = 1$.

In [21]:
values = {a: 3, r: 1}
fnk1 = sp.lambdify(u, k1.subs(values), 'numpy')
fnk2 = sp.lambdify(u, k2.subs(values), 'numpy')

# Plot 2D
u_values = np.linspace(0, 2*np.pi, 100)
k1_values = fnk1(u_values)
k2_values = fnk2(u_values)

fig = go.Figure()
fig.add_trace(go.Scatter(x=u_values, y=k1_values, name='k1'))
fig.add_trace(go.Scatter(x=u_values, y=k2_values, name='k2'))
fig.update_layout(title='Curvaturas principales', xaxis_title='u', yaxis_title='k')
fig.show()

- Evaluate and plot Gaussian curvature of the surface

In [38]:
fnk = sp.lambdify(u, K.subs(values), 'numpy')

u_values = np.linspace(0, 2*np.pi, 100)
k_values = fnk(u_values)

fig = go.Figure()
fig.add_trace(go.Scatter(x=u_values, y=k1_values, name='k1'))
fig.add_trace(go.Scatter(x=u_values, y=k2_values, name='k2'))
fig.add_trace(go.Scatter(x=u_values, y=k_values, name='Curvatura Gaussiana'))
fig.update_layout(title='Curvatura Gaussiana', xaxis_title='u', yaxis_title='k')
fig.show()

# 3. Consider the parametrization for the Möbius strip

$$\displaystyle \vec{x}(\theta, t) = \left(   \left(1 + \frac{1}{2}tcos(\frac{\theta}{2}) \right) cos(\theta),\;\; \left(1 + \frac{1}{2}tcos(\frac{\theta}{2}) \right) sin(\theta),\;\; \frac{1}{2}tsin(\frac{\theta}{2})  \right)$$

for 

$$\displaystyle U = \left\{ (\theta, t) \in R^{2} | -0.25 < t < 0.25,\;\; 0 < \theta < 2\pi  \right\}$$

In [23]:
theta, t    = sp.symbols('theta t', real = True)
tlimits     = (-0.25, 0.25)
thetalimits = (0, 2*sp.pi)

x = sp.Matrix([ (1 + 0.5*t*sp.cos(theta/2))*sp.cos(theta), (1 + 0.5*t*sp.cos(theta/2))*sp.sin(theta), 0.5*t*sp.sin(theta/2) ])
x

Matrix([
[(0.5*t*cos(theta/2) + 1)*cos(theta)],
[(0.5*t*cos(theta/2) + 1)*sin(theta)],
[                 0.5*t*sin(theta/2)]])

In [24]:
a_ = np.linspace(0, 2.0 * np.pi,30)
b_ = np.linspace(-0.25, 0.25, 5)
a_, b_ = np.meshgrid(a_, b_)
x_ = (1 + 0.5 * b_ * np.cos(a_ / 2.0)) * np.cos(a_)
y_ = (1 + 0.5 * b_ * np.cos(a_ / 2.0)) * np.sin(a_)
z_ = 0.5 * b_ * np.sin(a_ / 2.0)

fig = go.Figure(data=[go.Surface(x=x_, y=y_, z=z_, colorscale='Inferno')])
fig.update_layout(title='Möbius strip',
                  autosize=False,
                  width=500,
                  height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

- Evaluate the first fundamental form of the surface

In [25]:
# first fundamental form of the surface
xtheta = x.diff(theta)
xtheta

Matrix([
[-0.25*t*sin(theta/2)*cos(theta) - (0.5*t*cos(theta/2) + 1)*sin(theta)],
[-0.25*t*sin(theta/2)*sin(theta) + (0.5*t*cos(theta/2) + 1)*cos(theta)],
[                                                  0.25*t*cos(theta/2)]])

In [26]:
xt = x.diff(t)
xt

Matrix([
[0.5*cos(theta/2)*cos(theta)],
[0.5*sin(theta)*cos(theta/2)],
[           0.5*sin(theta/2)]])

In [27]:
E = xtheta.dot(xtheta).simplify()
F = xt.dot(xtheta).simplify()
G = xt.dot(xt).simplify()

E

-0.25*t**2*sin(theta/2)**2 + 0.3125*t**2 + 1.0*t*cos(theta/2) + 1.0

In [28]:
F

0

In [29]:
G

0.250000000000000

- Evaluate the metric tensor for the mobius strip

In [30]:
# calculate the metric tensors for the surface
g = sp.Matrix([[E, F], [F, G]])
g

Matrix([
[-0.25*t**2*sin(theta/2)**2 + 0.3125*t**2 + 1.0*t*cos(theta/2) + 1.0,    0],
[                                                                  0, 0.25]])

In [31]:
# determinant of the metric tensor, just for fun
g.det().simplify()

-0.0625*t**2*sin(theta/2)**2 + 0.078125*t**2 + 0.25*t*cos(theta/2) + 0.25

- Which is the area element $\| \vec{X}_{\theta} \times \vec{X}_{t} \|$ for the surface?

In [32]:
# Area element for the surface
ds = xtheta.cross(xt).norm().simplify()
ds

0.5*sqrt(0.25*(0.5*t*sin(theta) - 0.25*t*sin(2*theta) + sin(theta/2) - sin(3*theta/2))**2 + 0.25*(0.5*t*cos(theta/2)**2*cos(theta) + 0.25*t*cos(theta) - 0.375*t*cos(2*theta) + 0.125*t + cos(theta/2) - cos(3*theta/2))**2 + (-0.25*t*sin(theta/2)*sin(theta)*cos(theta) - 0.5*t*sin(theta)**2*cos(theta/2) + 0.75*t*cos(theta/2) - 0.0625*t*cos(3*theta/2) - 0.1875*t*cos(5*theta/2) + 1.0)**2*cos(theta/2)**2)

- Evaluate the total area for the ellipsoid as 

$$\int \int_{Q} \| \vec{X}_{\theta} \times \vec{X}_{t} \| d\theta dt$$

If the analytic solution is not possible, use numerical integration.

In [33]:
# lambdify the area element
ds_lambdify = sp.lambdify((theta, t), ds)

# calculate the integral with numeric integration
I = numeric_integration(ds_lambdify, [thetalimits, tlimits])
I

(1.5718231960534048, 3.494667781985225e-14)

- Obtain the normal vector as a function of the coordinates $(\theta, t)$

$$\vec{N}(\theta, t) = \frac{\vec{X}_{\theta} \times \vec{X}_{t}}{\| \vec{X}_{\theta} \times \vec{X}_{t} \|}$$

In [34]:
N = xtheta.cross(xt)/(xtheta.cross(xt).norm())
N

Matrix([
[                                                                            2.0*(-0.125*t*sin(theta)*cos(theta/2)**2 + 0.5*(-0.25*t*sin(theta/2)*sin(theta) + (0.5*t*cos(theta/2) + 1)*cos(theta))*sin(theta/2))/sqrt((0.25*t*sin(theta)*cos(theta/2)**2 + (0.25*t*sin(theta/2)*sin(theta) - (0.5*t*cos(theta/2) + 1)*cos(theta))*sin(theta/2))**2 + (0.25*t*cos(theta/2)**2*cos(theta) - (-0.25*t*sin(theta/2)*cos(theta) - (0.5*t*cos(theta/2) + 1)*sin(theta))*sin(theta/2))**2 + (-(-0.25*t*sin(theta/2)*sin(theta) + (0.5*t*cos(theta/2) + 1)*cos(theta))*cos(theta/2)*cos(theta) + (-0.25*t*sin(theta/2)*cos(theta) - (0.5*t*cos(theta/2) + 1)*sin(theta))*sin(theta)*cos(theta/2))**2)],
[                                                                             2.0*(0.125*t*cos(theta/2)**2*cos(theta) - 0.5*(-0.25*t*sin(theta/2)*cos(theta) - (0.5*t*cos(theta/2) + 1)*sin(theta))*sin(theta/2))/sqrt((0.25*t*sin(theta)*cos(theta/2)**2 + (0.25*t*sin(theta/2)*sin(theta) - (0.5*t*cos(theta/2) + 1)*cos(the

- Evaluate the normal vector at $t = 0$ and obtain the limits 

$$\lim_{\theta \to 0} \vec{N}(\theta, 0)$$

and 

$$\lim_{\theta \to 2\pi} \vec{N}(\theta, 0)$$

In [35]:
# limit of the normal vector as t = 0 and theta -> 0
Nlim1 = N.subs({t: 0}).limit(theta, 0)
Nlim1

Matrix([
[ 0],
[ 0],
[-1]])

In [36]:
# limit of the normal vector as t = 0 and theta -> 2*pi
Nlim2 = N.subs({t: 0}).limit(theta, 2*sp.pi)
Nlim2

Matrix([
[0],
[0],
[1]])